In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [4]:
norm_mean = torch.tensor([0.4914, 0.4822, 0.4465])
norm_std = torch.tensor([0.2470, 0.2435, 0.2616])
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [ ]:
for idx, (X, Y) in enumerate(training_dataloader):
    print(idx)
    print(X.shape, Y.shape)
    break

0
torch.Size([128, 3, 32, 32]) torch.Size([128])
